# Extra needed code

In [9]:
amino_acids={
    "A":71.037114,
    "R":156.101111,
    "N":114.042927,
    "D":115.026943,
    "C":103.009185,
    "E":129.042593,
    "Q":128.058578,
    "G":57.021464,
    "H":137.058912,
    "I":113.084064,
    "L":113.084064,
    "K":128.094963,
    "M":131.040485,
    "F":147.068414,
    "P":97.052764,
    "S":87.032028,
    "T":101.047679,
    "U":150.95363,
    "W":186.079313,
    "Y":163.06332,
    "V":99.068414
}

'''calc_masses

DESC:
    calculates the masses/spectrum for a sequence
Inputs:
    sequence: str amino acid sequence to change to list of masses
    charge: int charge value to calculate masses for
Outputs:
    list of floats, float       spectrum and the precursor mass 
'''
def calc_masses(sequence, charge):
    masses = []

    length = len(sequence)
    total = 2 * 1.007825035 + 15.99491463 #This is the mass of water. Adding the mass of water to the sum of all the residue masses gives the mass of the peptide.
    for i in range(length):
        total +=  amino_acids[sequence[i]]

    pre_mz = (total+charge*1.0072764)/charge   

    if charge == 1:
        #b+
        total = 1.007825035 - 0.0005486 #for the H to turn the residue NH on the N-terminus into NH2
        for i in range (0, length):
            total += amino_acids[sequence[i]]
            masses.append(total)
            #Since z (the charge) is equal to one, the total here is the m/z

        #y+
        total = 3 * 1.007825035 + 15.99491463 - 0.0005486 #for the OH to turn the residue CO on the C-terminus into COOH + 1 proton to make NH into NH2 and 1 proton make positively charged
        for i in range (0,length):
            total += amino_acids[sequence[length-i-1]]
            masses.append(total)

    elif charge == 2:
        #b++
        total = 2 * 1.007825035 - 2 * 0.0005486 #adding one more proton this time to make it doubly charged
        for i in range (0, length):
            total += amino_acids[sequence[i]]
            masses.append(total/2)

        #y++
        total = 4 * 1.007825035 + 15.99491463 - 2 * 0.0005486 #another proton to make doubly charged
        for i in range (0, length):
            total += amino_acids[sequence[length-i-1]]
            masses.append(total/2)
        #The masses you get exactly match Spectrum Mill. To get this, I had to make sure to use the mass of H+ and the mass of H when appropriate.

    return masses, pre_mz

In [24]:
def readfasta(fasta_file):
    prots = []
    with open(fasta_file, 'r') as i:
        name = None 
        seq = '' 
        identifier = ''
        for line in i:
            if '>' in line: #name line

                # add the last thing to the list
                if not ((name is None or name == '') and (seq is None or seq == '')):
                    prots.append({
                        'name': name,
                        'sequence': seq,
                        'identifier': identifier
                    })

                seq = '' 
                name = str(str(line.split('|')[2]).split(' ')[0]).replace('\n', '')
                identifier = str(line.split('|')[1])
            else:
                seq += line.replace('\n', '')
        # add the last one
        prots.append({
            'name': name,
            'sequence': seq,
            'identifier': identifier
        })
    return prots

In [25]:
from pyopenms import MSExperiment, MzMLFile

'''read

DESC:
    read an .mzML file into memory
Inputs:
    file: str path to the file to import
Outputs:
    NONE if file is not found

'''
def readmzml(file):

    spectra = []
    exp = MSExperiment()
    MzMLFile().load(file, exp)

    for s in exp.getSpectra():
        spectra.append({
            'level': s.getMSLevel(),
            'spectrum': list(list(s.get_peaks())[0]), 
            'scan_no': int(str(s.getNativeID()).split('=')[-1].replace("'", ''))
        })

    return spectra

# search and score code

In [28]:
def search_proteins(spectrum, db):
    best_match = {}
    for prot in db:
        score = cmp_string_spectra(prot['sequence'], spectrum['spectrum'])
        if best_match == {} or best_match['score'] < score:
            best_match = {
                'protein_id': prot['identifier'],
                'protein_name': prot['name'], 
                'protein_sequence': prot['sequence'], 
                'ms_level': spectrum['level'],
                'scan_no': spectrum['scan_no'], 
                'score': score
            }

    return best_match

In [22]:
def cmp_spectra_spectra(spec: list, reference: list) -> float:
    '''
    CREATED FEB 26 2020
    Score two spectra against eachother. Simple additive scoring with bonuses for streaks

    Inputs:
        spec:       list of floats (from mass spectra)
        reference:  list of floats (calculated from protein sequence)
    Outputs:
        score:      float score 
    '''
    if len(spec) == 0 or len(reference) == 0:
        return
    streak = 0
    last = False
    score = 0
    max_streak = 0
    for mass in spec:
        if last == True:
            streak += 1
            max_streak = max([streak, max_streak])

        if mass in reference:
            score += 1
            last = True

        else:
            streak = 0
            last = False
    
    score += max_streak
    return score 

In [15]:
def cmp_string_spectra(seq, ref_spec):
    spec1 = []
    m1, _ = calc_masses(seq, 1)
    m2, _ = calc_masses(seq, 2)
    spec1 = m1 + m2
    return cmp_spectra_spectra(spec1, ref_spec)

In [44]:
fastafile = '/Users/zacharymcgrath/Desktop/Experiment output/DEBUG OUTPUT/databases/peptide_9.fasta'
fastadb = readfasta(fastafile)
spectrafile_8 = '/Users/zacharymcgrath/Desktop/Experiment output/DEBUG OUTPUT/spectra/insulin_8.mzML'
spectrafile_20 = '/Users/zacharymcgrath/Desktop/Experiment output/DEBUG OUTPUT/spectra/insulin_20.mzML'
spectra8 = readmzml(spectrafile_8)
spectra20 = readmzml(spectrafile_20)

In [45]:
res8 = []
for s8 in spectra8:
    res8.append(search_proteins(s8, fastadb))

res20 = []
for s20 in spectra20:
    res20.append(search_proteins(s20, fastadb))

In [49]:
scored_r8 = [x for x in res8 if x['score'] > 0]
scored_r20 = [x for x in res20 if x['score'] > 0]

In [50]:
print([(x['scan_no'], x['score']) for x in scored_r8])
print([(x['scan_no'], x['score']) for x in scored_r20])

[(2, 3), (6, 6), (7, 3), (9, 24), (10, 10), (11, 4), (12, 13), (13, 6), (15, 3), (19, 24), (29, 3), (34, 3), (38, 3), (40, 3), (60, 3), (67, 3), (76, 3), (79, 6), (81, 3), (85, 3), (90, 3), (98, 3), (99, 3), (101, 3), (102, 3)]
[(2, 3), (6, 6), (7, 57), (8, 5), (9, 54), (10, 10), (11, 4), (12, 13), (13, 6), (15, 3), (29, 3), (34, 3), (38, 3), (40, 3), (60, 3), (67, 3), (76, 3), (79, 6), (81, 3), (85, 3), (87, 3), (90, 5)]


In [51]:
spec8_14 = [x for x in spectra8 if x ['scan_no'] == 14][0]
spec20_14 = [x for x in spectra20 if x ['scan_no'] == 14][0]

In [52]:
print(search_proteins(spec8_14, fastadb))

{'protein_id': 'id0', 'protein_name': 'peptide_9', 'protein_sequence': 'LLALLALWGPDPAAAFVN', 'ms_level': 2, 'scan_no': 14, 'score': 0}


In [53]:
print(search_proteins(spec20_14, fastadb))

{'protein_id': 'id0', 'protein_name': 'peptide_9', 'protein_sequence': 'LLALLALWGPDPAAAFVN', 'ms_level': 2, 'scan_no': 14, 'score': 0}


In [55]:
correct_score8 = [x for x in scored_r8 if x['scan_no'] == 9][0]
correct_score20 = [x for x in scored_r20 if x['scan_no'] == 9][0]
print(correct_score8)
print(correct_score20)

{'protein_id': 'id0', 'protein_name': 'peptide_9', 'protein_sequence': 'LLALLALWGPDPAAAFVN', 'ms_level': 2, 'scan_no': 9, 'score': 24}
{'protein_id': 'id0', 'protein_name': 'peptide_9', 'protein_sequence': 'LLALLALWGPDPAAAFVN', 'ms_level': 2, 'scan_no': 9, 'score': 54}
